In [1]:
import time
import copy
import numpy as np
import pandas as pd
import seaborn as sn
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.manifold import TSNE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

from models import *
from utils import *
from datasets import *

# ResNet

In [2]:
model = ResNet(BasicBlock, [1,1,1,1])
sum(p.numel() for p in model.parameters() if p.requires_grad)

321444

In [3]:
I_hat = torch.ones((1,10,1000,1000))

out = model(I_hat)
out.shape

torch.Size([1, 100])

In [2]:
model = ResNet(BasicBlock, [1,1,1,1])
sum(p.numel() for p in model.parameters() if p.requires_grad)

1254116

In [2]:
model = ResNet(BasicBlock, [1,1,1,1])
sum(p.numel() for p in model.parameters() if p.requires_grad)

4953444

# MM-II

In [4]:
model = AttnFusion(visual_size=100, metric_size=9)
visual = torch.ones((1,100))
metric = torch.ones((1,9))

feats = model(user_preference=metric, visual_feature=visual)
feats.shape

torch.Size([100])

# MMModel

In [13]:
model = MMModel(MM_I_wPATH="./results/encoder_weights.pt", MM_II_wPATH=None, freeze=(False, False), device=torch.device('cpu'))

param1 = sum(p.numel() for p in model.MM_I.parameters() if p.requires_grad)
param2 = sum(p.numel() for p in model.MM_II.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters() if p.requires_grad)

param1, param2, total

(2697986, 11277259, 13975245)

In [3]:
sum(p.numel() for p in model.MM_II.cnn.parameters() if p.requires_grad)

11224164

In [4]:
resnet = ResNet(BasicBlock, [1, 1, 1, 1])
sum(p.numel() for p in resnet.parameters() if p.requires_grad)

4953444

In [5]:
img = torch.ones((1,10,1000,1000))
out = resnet(img)
out.shape

torch.Size([1, 100])

# data

In [2]:
import torchvision as tv
import torchvision.transforms as transforms
from torch.autograd import Variable
from torchvision.utils import save_image

# Data Preprocessing

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
trainTransform  = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize((0.1307,), (0.3081,))])
trainset = tv.datasets.MNIST(root='./data',  train=True,download=True, transform=transform)
testset = tv.datasets.MNIST(root='./data',  train=False,download=True, transform=transform)

In [3]:
traindata = [i[0] for i in trainset]
trainlabel = [i[1] for i in trainset]
testdata = [i[0] for i in testset]
testlabel = [i[1] for i in testset]

X = traindata + testdata
y = trainlabel + testlabel

In [4]:
X = torch.vstack(X)
X.shape

torch.Size([70000, 28, 28])

In [5]:
X = X.unsqueeze(1)
X.shape

torch.Size([70000, 1, 28, 28])

# MM_II

In [18]:
# z, answers = model()

z = np.load("./tsne_data_reducted_normalised.npy")
z = torch.tensor(z)
z.shape

torch.Size([70000, 3])

In [19]:
def get_Ihat(Z):
    mat = np.zeros((1000,1000))
    for i in range(len(Z)):
        xx = min(int(np.floor(Z[i,0]*1000)), 999)
        yy = min(int(np.floor(Z[i,1]*1000)), 999)
        mat[xx,yy] = Z[i, 2]
    return mat

# VI = VisualImitation(device='cpu')
# I_hat = VI(z[:128,:])
I_hat = get_Ihat(Z=z)
I_hat = torch.tensor(I_hat).unsqueeze(0)
I_hat.shape

torch.Size([1, 1000, 1000])

In [20]:
model = HumanModel(cnn_layers=[2,2,2,2], metric_num=9, hidden_dim=100)
q1, q2, q3 = model(I_hat=torch.ones((1,10,1000,1000)), z=z[:1024,:2], labels=None, x=None)

./metrics/get_scag.r


In [21]:
q1, q2, q3

(tensor([[0.5342]], grad_fn=<SigmoidBackward0>),
 tensor([[0.4964]], grad_fn=<SigmoidBackward0>),
 tensor([[0.5380]], grad_fn=<SigmoidBackward0>))

# MMModel

In [6]:
model = MMModel(MM_I_wPATH="./results/encoder_weights.pt", MM_II_wPATH=None, freeze=(False, False), device=torch.device('cpu'))

In [10]:
VI = VisualImitation(device='cpu')
I_hat = VI(torch.ones((128,2)))
I_hat.shape

IndexError: index 2 is out of bounds for dimension 1 with size 2

In [7]:
q1, q2, q3 = model(x=X[:6], labels=torch.tensor(y[:6]).reshape(-1,1))

x: torch.Size([6, 1, 28, 28]), labels:torch.Size([6, 1])
z:torch.Size([6, 2])


IndexError: index 2 is out of bounds for dimension 1 with size 2